In [1]:
#Import libraries
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from numpy import arange
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing

In [2]:
dataset_train=pd.read_csv('file:///Users/mac/Downloads/Train.csv')
dataset_test=pd.read_csv('file:///Users/mac/Downloads/Test.csv')

In [3]:
## OUT OF THE 2 CATEGORICAL COLUMNS "NONE" has missing values
## i filled the missing values with a new categorical value "None"

## ALMOST ALL THE NUMERICAL COLUMNS HAVE MISSING VALUES except form_field14&19
## I FILL THE MISSING VALUES WITH A VALUE OF "0.0", you can use mean, median, mode values for this also (whichever works best)
for i in ['form_field1', 'form_field2', 'form_field3', 'form_field4', 'form_field5', 'form_field6', 'form_field7',
       'form_field8', 'form_field9', 'form_field10', 'form_field11', 'form_field12', 'form_field13', 'form_field15',
       'form_field16', 'form_field17', 'form_field18', 'form_field19', 'form_field20', 'form_field21', 'form_field22', 
       'form_field23', 'form_field24', 'form_field25', 'form_field26', 'form_field27', 'form_field28', 'form_field29',
       'form_field30', 'form_field31', 'form_field32', 'form_field33', 'form_field34', 'form_field35', 'form_field36',
       'form_field37', 'form_field38', 'form_field39', 'form_field40', 'form_field41', 'form_field42', 'form_field43',
       'form_field44','form_field45','form_field46','form_field48', 'form_field49', 'form_field50']:
    dataset_train[i] = dataset_train[i].fillna(value = 0.0)
dataset_train.isnull().sum()

Applicant_ID      0
form_field1       0
form_field2       0
form_field3       0
form_field4       0
form_field5       0
form_field6       0
form_field7       0
form_field8       0
form_field9       0
form_field10      0
form_field11      0
form_field12      0
form_field13      0
form_field14      0
form_field15      0
form_field16      0
form_field17      0
form_field18      0
form_field19      0
form_field20      0
form_field21      0
form_field22      0
form_field23      0
form_field24      0
form_field25      0
form_field26      0
form_field27      0
form_field28      0
form_field29      0
form_field30      0
form_field31      0
form_field32      0
form_field33      0
form_field34      0
form_field35      0
form_field36      0
form_field37      0
form_field38      0
form_field39      0
form_field40      0
form_field41      0
form_field42      0
form_field43      0
form_field44      0
form_field45      0
form_field46      0
form_field47      0
form_field48      0
form_field49      0


In [4]:
## OUT OF THE 2 CATEGORICAL COLUMNS "NONE" has missing values
## i filled the missing values with a new categorical value "None"

## ALMOST ALL THE NUMERICAL COLUMNS HAVE MISSING VALUES except form_field14&19
## I FILL THE MISSING VALUES WITH A VALUE OF "0.0", you can use mean, median, mode values for this also (whichever works best)
for i in ['form_field1', 'form_field2', 'form_field3', 'form_field4', 'form_field5', 'form_field6', 'form_field7',
       'form_field8', 'form_field9', 'form_field10', 'form_field11', 'form_field12', 'form_field13', 'form_field15',
       'form_field16', 'form_field17', 'form_field18', 'form_field20', 'form_field21', 'form_field22', 
       'form_field23', 'form_field24', 'form_field25', 'form_field26', 'form_field27', 'form_field28', 'form_field29',
       'form_field30', 'form_field31', 'form_field32', 'form_field33', 'form_field34', 'form_field35', 'form_field36',
       'form_field37', 'form_field38', 'form_field39', 'form_field40', 'form_field41', 'form_field42', 'form_field43',
       'form_field44','form_field45','form_field46','form_field48', 'form_field49', 'form_field50']:
    dataset_test[i] = dataset_test[i].fillna(value = 0.0)
dataset_test.isnull().sum()

Applicant_ID    0
form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [16]:
X = dataset_train.drop(["default_status"], axis=1)
y = dataset_train['default_status']

X_test = dataset_test.drop(["Applicant_ID", "form_field47"], axis = 1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=27)

print(y_train.value_counts())
print(y_test.value_counts())

no     29666
yes     9534
Name: default_status, dtype: int64
no     12619
yes     4181
Name: default_status, dtype: int64


In [21]:
    #provide categorical features to catboost
cat_features = ['Applicant_ID','form_field47']

best_params = {
        'bagging_temperature': 0.5,
        'depth': 8,
        'iterations': 1000,
        'l2_leaf_reg': 25,
        'learning_rate': 0.05,
        'sampling_frequency': 'PerTreeLevel',
        'leaf_estimation_method': 'Gradient',
        'random_strength': 0.8,
        'boosting_type': 'Ordered',
        'feature_border_type': 'MaxLogSum',
        'l2_leaf_reg': 50,
        'max_ctr_complexity': 2,
        'fold_len_multiplier': 2
    }

model_cat = CatBoostClassifier(**best_params,
                            loss_function='Logloss',
                            eval_metric='AUC',
                            nan_mode='Min',
                            thread_count=8,
                            task_type='CPU',
                            verbose=True)


model_cat.fit(X_train, y_train,
                            eval_set=(X_test, y_test),
                            cat_features=cat_features,
                            verbose_eval=300,
                            early_stopping_rounds=500,
                            use_best_model=True,
                            plot=False)


model_cat.save_model("catmodel")

##Predictions
cat_predictions = model_cat.predict_proba(dataset_test)[:, 1]
cat_predictions_df = pd.DataFrame({'Applicant_ID':dataset_test['Applicant_ID'],'default_status': cat_predictions})

0:	test: 0.7993075	best: 0.7993075 (0)	total: 520ms	remaining: 8m 39s
300:	test: 0.8375982	best: 0.8375982 (300)	total: 3m 6s	remaining: 7m 13s
600:	test: 0.8366144	best: 0.8376644 (324)	total: 6m 8s	remaining: 4m 4s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8376644301
bestIteration = 324

Shrink model to first 325 iterations.


In [22]:
#submission = pd.DataFrame({'Applicant_ID':dataset_test['Applicant_ID'],'default_status':np.mean(y_pred_totcb,0)})
#submission.head()
cat_predictions_df.head()


#submission.to_csv('cb20foldscatMostfeat22Opt_U2.csv',index=False)
cat_predictions_df.to_csv('Model_Loan_Default.csv', index=False)